In [ ]:
data_raw = sc.textFile('data/201508_trip_data.csv.gz')

In [ ]:
data_raw.take(5)

In [ ]:
from pyspark.sql import Row

In [ ]:
durations = spark.createDataFrame(data_raw
             .filter(lambda x: not x.startswith('Trip'))
             .map(lambda x: x.split(','))
             .map(lambda x: Row(trip_id = int(x[0]), duration = int(x[1]))))

In [ ]:
from pyspark.sql.functions import mean, stddev, udf
from pyspark.sql.types import FloatType

In [ ]:
stats = durations.select(stddev(durations.duration).alias('sd'),
                         mean(durations.duration).alias('mu')).collect()

In [ ]:
sd, mu = stats[0]

In [ ]:
z_vals = durations.select(durations.trip_id,
                 udf(lambda x: (x - mu)/sd, FloatType())('duration').alias('z'))

In [ ]:
import numpy as np

In [ ]:
zs = np.array([i[0] for i in z_vals.select('z').collect()])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.hist(zs[zs<5],bins=30)
plt.yscale('log')
None

In [ ]:
dates = (data_raw
         .map(lambda x: x.split(','))
         .map(lambda x: x[2]))

In [ ]:
dates.take(5)

In [ ]:
from datetime import datetime

In [ ]:
def convert_date(date_str):
    try:
        return datetime.strptime(date_str, '%m/%d/%Y %H:%M')
    except ValueError:
        return None

In [ ]:
dates = (dates
        .map(lambda x: convert_date(x))
        .filter(lambda x: x != None))

In [ ]:
dates.take(5)

In [ ]:
from operator import add

In [ ]:
def seqOp(d, (h, c)):
    d[h] = c
    return d
def combOp(d1, d2):
    d1.update(d2)
    return d1

In [ ]:
hourly_counts = (
    dates
    .map(lambda x: ((x.year, x.month, x.day, x.hour), 1))
    .reduceByKey(add)
    .map(lambda ((y, m, d, h), c): ((y, m, d), (h, c)))
    .aggregateByKey({}, seqOp, combOp))

In [ ]:
from pyspark.ml.linalg import Vectors

In [ ]:
X = (hourly_counts
     .map(lambda (day, d): (day, Vectors.dense([d.get(i, 0) for i in range(24)]))))

In [ ]:
X.cache()

In [ ]:
from pyspark.sql import Row

In [ ]:
Xdf = spark.createDataFrame(X.map(lambda x: Row(features = x[1], day=x[0])))

In [ ]:
from pyspark.ml.clustering import KMeans

In [ ]:
X_norm.cache()
Xdf.cache()

In [ ]:
ks = range(2, 10)
costs = []
for k in ks:
    costs.append(KMeans().setK(k).fit(Xdf).computeCost(Xdf))

In [ ]:
plt.plot(ks, costs)

In [ ]:
model = KMeans().setK(4).fit(Xdf)

In [ ]:
centers = [Vectors.dense(x) for x in model.clusterCenters()]

In [ ]:
predictions = model.transform(Xdf)
predictions.show(5)

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType, DateType, IntegerType, StringType
from pyspark.ml.linalg import VectorUDT

In [ ]:
from calendar import day_abbr, month_abbr
from datetime import date
def get_weekday(d):
    return day_abbr[date(*d).weekday()]

In [ ]:
per_weekday = predictions.select(udf(get_weekday, StringType())(predictions.day).alias('weekday'),
                  predictions.prediction.alias('class'))

In [ ]:
per_weekday.show(5)

In [ ]:
per_weekday.groupby('class', 'weekday').count().orderBy(per_weekday['class']).show()

In [ ]:
df = per_weekday.groupby('class', 'weekday').count().orderBy(per_weekday['class']).toPandas()

In [ ]:
df.head()

In [ ]:
df.pivot_table(index='class', columns='weekday', values='count').fillna(0).plot.bar()

In [ ]:
per_month = predictions.select(udf(lambda x: month_abbr[x[1]],
                                   StringType())(predictions.day).alias('month'),
                  predictions.prediction.alias('class'))

In [ ]:
mdf = per_month.groupBy('month', 'class').count().toPandas()

In [ ]:
mdf.head()

In [ ]:
mdf.pivot_table(index='class', values='count', columns='month').plot.bar()
plt.legend(bbox_to_anchor=(1,1))

In [ ]:
for n, center in enumerate(centers):
    plt.plot(center, label='class {}'.format(n))
plt.legend()

In [ ]:
def dist_from_center(x, cluster):
    return float((x - centers[cluster]).norm(2))

In [ ]:
d_udf = udf(dist_from_center, DoubleType())

In [ ]:
dists = predictions.select(d_udf(predictions['features'], predictions['prediction']).alias('dist'),
                           predictions.day, predictions.prediction)

In [ ]:
dists.orderBy(dists.dist.desc()).show(5)

In [ ]:
from pyspark.sql.functions import stddev, mean

In [ ]:
dist_stats = dists.groupby('prediction').agg(stddev('dist').alias('sd'), mean('dist').alias('mu'))

In [ ]:
dist_stats.show()

In [ ]:
joined = dists.join(dist_stats, dist_stats.prediction == dists.prediction).drop(dist_stats.prediction)
joined.show()

In [ ]:
z_vals = joined.select(udf(lambda d, mu, sd: abs(d - mu)/sd, FloatType())(
    joined.dist, joined.mu, joined.sd).alias('z'), joined.day, "prediction")

In [ ]:
z_vals.orderBy(z_vals.z.desc()).show(5)

In [ ]:
for day, features, cluster in predictions.rdd.filter(lambda x :x.day in (Row(2015,6,1),
                                                                         Row(2014,10,20))).collect():
    plt.plot(features, label='outlier: {}'.format(day))
plt.plot(centers[cluster], label='cluster mean')
plt.legend(loc='lower left')
plt.xlabel('hour')
plt.ylabel('trip count')